In [1]:
#ignition delay for RMG model of propane 
import pandas as pd
import numpy as np
import time
import cantera as ct
import sys 

In [2]:
print(f"Runnning Cantera version: {ct.__version__}")

#constants
pressure = ct.one_atm 
mole_frac_dict = {"C3H8(1)": 1, "O2(2)": 5, "Ar":94} #this technically sets equivalence ratio to 1 
#mole_frac_dict_eduardos = {"C3H8(1)": 1, "O2(2)": 6, "Ar":93}

def ignition_delay(states, species): 
    """
    This function computes the ignition delay from the occurence of the peak in species' concentration
    """
    # pd = states.to_pandas()
    # print(pd)
    i_ign = states(species).Y.argmax()
    return states.t[i_ign]

def gas_conditions(temp): 
    """
    define gas conditions
    """
    gas.TPX = temp, pressure, mole_frac_dict

Runnning Cantera version: 2.5.1


In [3]:
#to save data
taus = []


reference_species = "OH(6)"

x_axis_vals= list(np.linspace(0.55, 0.8, 20))


for i in x_axis_vals: 
        
    gas = ct.Solution("./cantera/chem_annotated.yaml")
    temp= 1000/i
    gas_conditions(temp)
    
    # Create a batch reactor object and add it to a reactor network
    # In this example, the batch reactor will be the only reactor in the network
    r = ct.IdealGasConstPressureReactor(contents=gas, name="Batch Reactor")
    reactor_network = ct.ReactorNet([r])
    
    
    # use the above list to create a DataFrame
    time_history = ct.SolutionArray(gas, extra="t")

    t0 = time.time()
    
    # This is a starting estimate. If you do not get an ignition within this time, increase it
    estimated_ignition_delay_time = 0.4
    t = 0

    counter = 1
    while t < estimated_ignition_delay_time:
        t = reactor_network.step()
        if not counter % 10:
            # We will save only every 10th value. Otherwise, this takes too long
            # Note that the species concentrations are mass fractions
            time_history.append(r.thermo.state, t=t)
        counter += 1

    # We will use the 'OH(6)' species to compute the ignition delay
    tau = ignition_delay(time_history, reference_species)

    # Toc
    t1 = time.time()

    print(f"Computed Ignition Delay: {tau:.3e} seconds. Took {t1-t0:3.2f}s to compute")

    #save to large data array
    taus.append(tau)
    
    # If you want to save all the data - molefractions, temperature, pressure, etc
    #time_history.to_pandas().to_csv("time_history.csv")


Computed Ignition Delay: 7.199e-05 seconds. Took 0.27s to compute
Computed Ignition Delay: 8.140e-05 seconds. Took 0.28s to compute
Computed Ignition Delay: 8.681e-05 seconds. Took 0.29s to compute
Computed Ignition Delay: 8.819e-05 seconds. Took 0.29s to compute
Computed Ignition Delay: 1.053e-04 seconds. Took 0.30s to compute
Computed Ignition Delay: 1.216e-04 seconds. Took 0.29s to compute
Computed Ignition Delay: 1.295e-04 seconds. Took 0.33s to compute
Computed Ignition Delay: 1.542e-04 seconds. Took 0.33s to compute
Computed Ignition Delay: 1.895e-04 seconds. Took 0.35s to compute
Computed Ignition Delay: 2.488e-04 seconds. Took 0.30s to compute
Computed Ignition Delay: 2.995e-04 seconds. Took 0.32s to compute
Computed Ignition Delay: 3.882e-04 seconds. Took 0.33s to compute
Computed Ignition Delay: 5.131e-04 seconds. Took 0.35s to compute
Computed Ignition Delay: 6.817e-04 seconds. Took 0.35s to compute
Computed Ignition Delay: 9.248e-04 seconds. Took 0.37s to compute
Computed I

In [48]:
#save to csv file 
dataframe = pd.DataFrame([x_axis_vals, taus]) 
dataframe.to_csv('C3H8_ignition_delays_eds_stoic.csv', index=False)

In [4]:
#save to csv file 
dataframe = pd.DataFrame([x_axis_vals, taus]) 
dataframe.to_csv('C3H8_ignition_delays.csv', index=False)

In [6]:
nora_df = pd.read_csv('/work/westgroup/nora/Code/projects/PFAS/models/C3H8_CH2F2/C3H8/C3H8_davids_hydrocarbon_settings/C3H8_ignition_delays.csv')
nora_df.iloc[1].values

array([7.19926468e-05, 8.13955442e-05, 8.68062153e-05, 8.81881788e-05,
       1.05272997e-04, 1.21615542e-04, 1.29510221e-04, 1.54221526e-04,
       1.89503382e-04, 2.48809438e-04, 2.99521189e-04, 3.88152202e-04,
       5.13121909e-04, 6.81738712e-04, 9.24808824e-04, 1.29227589e-03,
       1.80304674e-03, 2.55915423e-03, 3.64705339e-03, 5.18697249e-03])